# Liquid Instrument Data Logger Example

To demonstrates use of the Datalogger instrument to log time-series voltage data to a (Binary or CSV) file.

## Setup the Instrument

Include the library and create the instrument object

In [ ]:
import os
import time

from moku.instruments import Datalogger

### Connect the instrument
Connect to your Moku by its ip address ('192.168.###.###' or ipv6 addr) using Datalogger

In [ ]:
i = Datalogger('[fe80::7269:79ff:feb0:a1c%10]', force_connect=False)

### Configure the frontend

In [ ]:
#i.set_frontend(channel=1, impedance='1MOhm', coupling="DC", range="10Vpp")
i.set_samplerate(1000) # Log 1000 samples per second
i.set_acquisition_mode(mode='Precision')

## Aquire Data


In [ ]:
# Stop an existing log, if any, then start a new one. 10 seconds of both channels.
logFile = i.start_logging(duration=10)

# Track progress percentage of the data logging session
is_logging = True
while is_logging:
    # Wait for the logging session to progress by sleeping 0.5sec
    time.sleep(0.5)
    # Get current progress percentage and print it out
    progress = i.logging_progress()
    remaining_time = int(progress['time_to_end'])
    is_logging = remaining_time > 1
    print(f"Remaining time {remaining_time} seconds")

Log time-series voltage data to a file

In [ ]:
# Download log from Moku, use liconverter to convert this .li file to .csv
i.download("persist", logFile['file_name'], os.path.join(os.getcwd(), logFile['file_name']))
print("Downloaded log file to local directory.")

In [ ]:
import matplotlib.pyplot as plt

# stream the data for 10 seconds..
i.start_streaming(10)

# Set up the plotting parameters
plt.ion()
plt.show()
plt.grid()
plt.ylim([-1, 1])

line1, = plt.plot([])

# Configure labels for axes
ax = plt.gca()

# This loops continuously updates the plot with new data
while True:
    # get the chunk of streamed data
    data = i.get_stream_data()
    if data:
        plt.xlim([data['time'][0], data['time'][-1]])
        # Update the plot
        line1.set_ydata(data['ch1'])
        line1.set_xdata(data['time'])
        plt.pause(0.001)

## Close the Instrument

In [30]:
i.relinquish_ownership()